In [5]:
import pandas as pd
import heapq

# Metabolic Network

## Description of the approach taken to solve the problems

Exercise 1 and exercise 2 have been solved in a very similar way. Moreover, both used the same input strucutre to build the metabolic network (CSV file).

The aim of exercise 1 was identify the **non essential** enzymes, while exercise2 focuses on finding the **essential** enzymes. Both tasks run the BFS algorithm to determine the shortest path, utilizing the given input metabolites and biological endpoint. In exercise 3, the objective shifts to identify enzymes that are suitable to remove in order to kill the cancerous cells. For that task I implement the Dijkstra algorithmn in order to find the healthiest path.

For Exercise 1, identifying redundant enzymes involves extracting enzymes that the BFS has not traversed, i.e., those not visited during the algorithm's execution. To achieve this, we subtract the enzymes visited by the BFS from the enzymes in the given metabolic network.

In Exercise 2, the goal is to display the enzymes obtained during the BFS execution.

In exercise 3, identifying enzymes suitable for removal to eliminate cancerous cells, involved find a path where the enzymes are the most healty one. To achieve this, the Dijkstra algorithm was employed, utilizing health ratios as weights to find the healthiest and essential path. Enzymes not traversed by Dijkstra were deemed redundant and targeted for removal if were affected for cancerous cells. Those ones are the enzymes  output in the exercise.

In [3]:
def build_graph(filename):
    df = pd.read_csv(filename)
    graph = {}
    for index, row in df.iterrows():
        from_node = row['From']
        to_node = row['To']
        enzyme = row['Enzyme']

        if from_node not in graph:
            graph[from_node] = []

        graph[from_node].append((enzyme, to_node))

    return graph

In [41]:
def bfs(graph, start_node, end_node):
    queue = []
    queue.append([(start_node,"")])
    while queue:
        path = queue.pop(0)
        node = path[-1]
        if node[0] == end_node:
            return path
        
        for adjacent in graph.get(node[0], []):
            new_path = list(path)
            new_path.append((adjacent[1],adjacent[0]))
            queue.append(new_path)

### Exercise 1: 
**Using the simplified representation of glycolysis in Figure 1, write code that identifies all of the NON-ESSENTIAL enzymes in glycolysis. Note that this code should be able to generalise to other types of pathways with a single biological endpoint (e.g. glucose -> pyruvate).**

In [40]:
###### INPUT DATA #######
filename = "glyc.csv"
start_node = "Glucose"
end_node = "Pyruvate"


###### Exercise 1 #######
graph1 = build_graph(filename)
enzymes_graph = {enzyme[0] for enzymes in graph1.values() for enzyme in enzymes}

path = bfs(graph1, start_node, end_node)
enzymes_path ={enzyme[1] for enzyme in path}

unvisited = enzymes_graph - enzymes_path ## Redundant enzymes

print(unvisited)

{'Triosephosphate isomeras'}


### Exercise 2:
**The network in Figure 2 is given here as an edge-list. Modify your code from task 1 to identify all of the networks ESSENTIAL enzymes (enzymes that can NOT be removed from the network in order for the network to be able to produce ALL of its biological endpoints).**

In [50]:
###### INPUT DATA #######
filename = "ccm.csv"
start_node = "Glucose [c]"
end_node = "Glycine"


###### Exercise 2 #######
graph2 = build_graph(filename)          
            
path = bfs(graph2, start_node, end_node)
enzymes_path ={enzyme[1] for enzyme in path[1:]}

print(enzymes_path)

{'Enzyme1', 'Enzyme2', 'Enzyme4', 'PseudoEnzymes', 'Enzyme8', 'Enzyme6', 'Enzyme3'}


### Exercise 3:

**Identify enzymes that are suitable to target in order to kill the cancer-cells, but not the patients healthy cells? You can assume that the drug perfectly inhibits the enzyme, stopping all activity. Complete absence of transcripts (RNA molecules) indicates that the enzyme is not expressed at all.**

In [66]:
###### INPUT DATA #######
weights_file = "RNAcounts.csv"
graph2_file = "ccm.csv"

start_node = "Glucose [c]"
end_node = "Glycine"

###### Exercise 3 #######
#### Building the weighted graph
def build_weights(filename):
    df = pd.read_csv(filename)
    weights = {}
    for index, row in df.iterrows():
        enzyme = row["Enzyme"]
        cancer = row["Cancer RNA count"]
        health = row["Healthy RNA count"]
        weights[enzyme] = (health if cancer == 0 else health/cancer, health, cancer)
    return weights

def add_graph_weights(graph, weights):
    for enzymes in graph.items():
        for idx, enzyme in enumerate(enzymes[1]):
            tmp = graph[enzymes[0]][idx][0]
            w = weights[tmp]
            graph[enzymes[0]][idx] =  (w,) + graph[enzymes[0]][idx]      
    return graph

            
#### Dijkstra algorithmn
def dijkstra(graph, start_node, end_node):
    priority_queue = [(0, start_node, [])]

    while priority_queue:
        (cost, current_node, path) = heapq.heappop(priority_queue)

        if current_node == end_node:
            return path

        for adjacent in graph.get(current_node, []):
            neighbor = adjacent[2]
            edge_weight = adjacent[0][0]
            heapq.heappush(priority_queue, (cost + edge_weight, neighbor, path + [(edge_weight, adjacent[1], neighbor)]))


#### Solving the problem
weights = build_weights(weights_file)
graph2 = build_graph(graph2_file)
graph2_w = add_graph_weights(graph2, weights)

enzymes_graph = {enzyme[1] for enzymes in graph2_w.values() for enzyme in enzymes}

path = dijkstra(graph2_w,start_node,end_node)
enzymes_path ={enzyme[1] for enzyme in path}

unvisited = enzymes_graph - enzymes_path ## Redundant enzymes
benign_enzym = [enzym for enzym in unvisited if weights[enzym][2] != 0]

print(benign_enzym)

['Enzyme19', 'Enzyme23', 'Enzyme33', 'Enzyme21', 'Enzyme11', 'Enzyme32', 'Enzyme10', 'Enzyme20', 'Transporter2', 'Enzyme4', 'Enzyme27', 'Enzyme28', 'TKT', 'Enzyme12', 'Enzyme29', 'Enzyme16', 'Enzyme3', 'Enzyme7', 'Enzyme31', 'Enzyme25', 'Enzyme34', 'Enzyme17', 'Enzyme9', 'Enzyme26', 'Transporter1', 'Enzyme22', 'Enzyme24', 'Enzyme18', 'Enzyme5', 'Enzyme35']
